In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import re
from acquire import extract, build_extracted_df, get_new_patch_data, get_players
from prepare import prepare
from env import api_key

In [38]:
df = pd.read_csv('new_extracted_data_smith_time_15.csv')

In [39]:
len(df)

2389

In [40]:
df = df[2249:]

In [41]:
df = clean(df)

2249 1
2249 2
2249 3
2249 4
2249 5
2250 1
2250 2
2250 3
2250 4
2250 5
2251 1
2251 2
2251 3
2251 4
2251 5
2252 1
2252 2
2252 3
2252 4
2252 5
2253 1
2253 2
2253 3
2253 4
2253 5
2254 1
2254 2
2254 3
2254 4
2254 5
2255 1
2255 2
2255 3
2255 4
2255 5
2256 1
2256 2
2256 3
2256 4
2256 5
2257 1
2257 2
2257 3
2257 4
2257 5
2258 1
2258 2
2258 3
2258 4
2258 5
2259 1
2259 2
2259 3
2259 4
2259 5
2260 1
2260 2
2260 3
2260 4
2260 5
2261 1
2261 2
2261 3
2261 4
2261 5
2262 1
2262 2
2262 3
2262 4
2262 5
2263 1
2263 2
2263 3
2263 4
2263 5
2264 1
2264 2
2264 3
2264 4
2264 5
2265 1
2265 2
2265 3
2265 4
2265 5
2266 1
2266 2
2266 3
2266 4
2266 5
2267 1
2267 2
2267 3
2267 4
2267 5
2268 1
2268 2
2268 3
2268 4
2268 5
2269 1
2269 2
2269 3
2269 4
2269 5
2270 1
2270 2
2270 3
2270 4
2270 5
2271 1
2271 2
2271 3
2271 4
2271 5
2272 1
2272 2
2272 3
2272 4
2272 5
2273 1
2273 2
2273 3
2273 4
2273 5
2274 1
2274 2
2274 3
2274 4
2274 5
2275 1
2275 2
2275 3
2275 4
2275 5
2276 1
2276 2
2276 3
2276 4
2276 5
2277 1
2277 2
2277 3

ValueError: cannot convert float NaN to integer

In [37]:
def clean(df):
    '''
    This Function, fills NA with zero, creates team total columns & gold difference, drops killplayers_0
    '''

    # Drop Duplicates
    df = df.drop_duplicates()
    
    # Replace all nulls with 0
    df = df.fillna(0)

    # Setting the features we want to add up for team totals to 'columns'
    columns=['deathsplayer_',
                'goldPerSecond_',
                'jungleMinionsKilled_',
                'killsplayer_',
                'level_',
                'magicDamageDoneToChampions_',
                'minionsKilled_',
                'physicalDamageDoneToChampions_',
                'timeEnemySpentControlled_',
                'totalDamageDoneToChampions_',
                'totalGold_',
                'trueDamageDoneToChampions_',
                'ward_player_',
                'assistsplayer_',
                'xp_']

    # Initializing/Set up the names based on columns above
    for col in columns:
        df[f'team_{col}100'] = 0
        df[f'team_{col}200'] = 0

    # Calculate values for blue team totals and read team totals
    for index, value in enumerate(df.iterrows()):
        for col in columns:
            
            total = 0
            # Blue Team
            for i in range(1, 6):
                total += int(df.iloc[index][f'{col}{i}'])
            
            df.at[index, f'team_{col}100'] = total
        
            total = 0
            # Red Team
            for i in range(6, 11):
                total += int(df.iloc[index][f'{col}{i}'])
            
            df.at[index, f'team_{col}200'] = total

    # Drop killsplayer_0, not needed
    df.drop(columns = ['killsplayer_0'], inplace = True)

    # Rename team stats 100 = Blue Team, 200 Red Team
    df = df.rename(columns={ 'team_deathsplayer_100':'BlueTeamDeaths', 'team_deathsplayer_200':'RedTeamDeaths', 'team_goldPerSecond_100':'BlueTeamGoldPerSec', 'team_goldPerSecond_200':'RedTeamGoldPerSec', 'team_jungleMinionsKilled_100':'BlueTeamJungleMinionsKilled', 'team_jungleMinionsKilled_200':'RedTeamJungleMinionsKilled', 'team_killsplayer_100':'BlueTeamKills', 'team_killsplayer_200':'RedTeamKills', 'team_level_100':'BlueTeamLevel', 'team_level_200':'RedTeamLevel', 'team_magicDamageDoneToChampions_100':'BlueTeamMagicDamageDoneToChampions', 'team_magicDamageDoneToChampions_200':'RedTeamMagicDamageDoneToChampions', 'team_minionsKilled_100':'BlueTeamMinionsKilled', 'team_minionsKilled_200':'RedTeamMinionsKilled', 'team_physicalDamageDoneToChampions_100':'BlueTeamPhysicalDamageDoneToChampions', 'team_physicalDamageDoneToChampions_200':'RedTeamPhysicalDamageDoneToChampions', 'team_timeEnemySpentControlled_100':'BlueTeamTimeEnemySpentControlled', 'team_timeEnemySpentControlled_200':'RedTeamTimeEnemySpentControlled', 'team_totalDamageDoneToChampions_100':'BlueTeamTotalDamageDoneToChampions', 'team_totalDamageDoneToChampions_200':'RedTeamTotalDamageDoneToChampions', 'team_totalGold_100':'BlueTeamTotalGold', 'team_totalGold_200':'RedTeamTotalGold', 'team_trueDamageDoneToChampions_100':'BlueTeamTrueDamageDoneToChampions', 'team_trueDamageDoneToChampions_200':'RedTeamTrueDamageDoneToChampions', 'team_ward_player_100':'BlueTeamWards', 'team_ward_player_200':'RedTeamWards', 'team_assistsplayer_100':'BlueTeamAssists', 'team_assistsplayer_200':'RedTeamAssists', 'team_xp_100':'BlueTeamXp', 'team_xp_200':'RedTeamXp'})
    
    # Create Gold Difference
    df['BlueTeamTotalGoldDifference'] = df.BlueTeamTotalGold - df.RedTeamTotalGold
    df['RedTeamTotalGoldDifference'] =  df.RedTeamTotalGold - df.BlueTeamTotalGold

    # Creating BlueTeamMVPKills
    for index, value in enumerate(df.iterrows()):
        col='killsplayer_'
                
        # Blue Team MVP
        total = 0

        for i in range(1, 6):
            print(index+2249, i)
            value = int(df.iloc[index][f'{col}{i}'])
            if (value>total):
                total = value
    
        df.at[index, 'BlueTeamMVPKills'] = total
        
        # Red Team MVP
        total = 0

        for i in range(6, 11):
            value = int(df.iloc[index][f'{col}{i}'])
            if (value>total):
                total = value
            
        df.at[index, 'RedTeamMVPKills'] = total

    return df

In [12]:
train.shape, test.shape

((1951, 231), (488, 231))